In [1]:
import tensorflow as tf
import numpy as np
import scipy.linalg as ln
from model.ntm_ops import *
%load_ext autoreload
%autoreload 2

In [2]:
shift_conv = ln.circulant(np.arange(20)).T[
            np.arange(-(3 // 2), (3 // 2) + 1)
        ][::-1]

In [6]:
sess = tf.InteractiveSession()

In [11]:
def difference(x, y):
    return np.mean(np.sum(np.square(x - y), axis=1))

## Introduction
Neural Turing Machines combined the ability of Turing Machine and Neural Networks to infer simple algorithms. The controller (it's usually a LSTM) can be viewed as CPU and the external memory can be seen as RAM. 

A NTM has four components: Controller, read heads, write heads, and an external memory. 

High level overview:
1. Addressing: Addressing mechanism is used to produce the weightings of each head. There are two types of adrressing, content based and location based. At every time step, the controller outputs five elements to produce weightings of each head: key vector, key strength, interpolation gate, shift weighting, and a scalar that used to sharpen the weightings. 
2. Read: each read head has a weighting vector tells how much degree of information we read from on each memory location
3. Write: each write head has a weighting vector, an erase vector and an add vector. This is inspired by LSTM's forget gate and input gate. 

## Section 1 Hyper parameters

### 1.1 Memory matrix
Define two hyper parameters for the memory matrix: $N \times M$, where $N$ is the number of memory locations, $M$ is the vector size at each memory location

In [3]:
# N memory locations and each has M elements
N, M = 128, 20
# batch size
batch_size = 1

### 1.2 Controller dimension
Define the LSTM hidden state dimension h and stacked hidden layer number a. This is the same as tradition LSTM with the hidden state and cell state.

Define the output and input dimension, in NTM, it usually is how many bits per sequence. e.g. If one of the input sequence is [0, 1, 0, 1, 0, 1], then it should be 6.

In [4]:
a, h = 1, 100
input_dim = 8

### 1.3 The range of allowed location shift
Define the range of the allowed location shift in location based addressing (Convolutional shift), s. e.g. if s = 3, then allowed location shift will be [-1, 0, 1]

In [5]:
s = 3

## Section 2 Controller (LSTM)
At every time step the controller outputs weighting of each head and hidden states(including cell states in original LSTM).. The weighting is determined by addressing mechanism:
1. Content Addressing
2. Interpolation
3. Convolutional Shift
4. Sharpening

In [50]:
# 1. content based addressing, cosine similarity test
key_vector = tf.constant(dtype=tf.float32, value=np.array([
            [3, 4, 10, 11]
        ]))
memory = tf.constant(dtype=tf.float32, value=np.array([
            [
                [6, 8, 12, 0],
                [1, 2, 2, 3],
                [2, 3, 4, 10],
                [2, 3, 2, 10],
                [2, 3, 6, 0],
                [12, 23, 4, 10]
            ]
        ]))
result = sess.run(cosine_similarity(key_vector, memory))
# print "difference", difference(np.array([[ 1.        ,  0.98386991,  0.99846041]]), result)
# 2. content based addressing 
key_strength = tf.constant(value=3, shape=(1, 1), dtype=tf.float32)
result = tf.mul(key_strength, result)
w = tf.nn.softmax(result, name="content_weighting")
w = sess.run(w)

Tensor("content_weighting_13:0", shape=(1, 6), dtype=float32)


In [69]:
def shift(w_gt, s_t):
    """
    Perform the shifting operation as described in equation 8 from the paper.
    """

    # This function could be more performant.
    N = w_gt.size

    backward = [1, 0, 0]
    same     = [0, 1, 0]
    forward  = [0, 0, 1]
    null     = [0, 0, 0]

    restrictionList = []
    for i in range(0, N):
        if i == 0:
            restrictionList.append(backward)
        elif i == 1:
            restrictionList.append(same)
        elif i == N-1:
            restrictionList.append(forward)
        else:
            restrictionList.append(null)
    rT = np.array(restrictionList)

    if N >= 3:
        s_t = np.dot(rT, s_t)
    print s_t, rT
    sums = []
    for i in range(N):
        sums.append(0)
        for j in range(N):
            print sums, i, (i - j) % N, i - j
            sums[i] += w_gt[j] * s_t[(i - j) % N]

    return np.array(sums)


In [72]:
shift(np.array([1, 2, 3, 4, 5, 6]), np.array([0.1 ,0.3, 0.5]))

[ 0.1  0.3  0.   0.   0.   0.5] [[1 0 0]
 [0 1 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 1]]
[0] 0 0 0
[0.10000000000000001] 0 5 -1
[1.1000000000000001] 0 4 -2
[1.1000000000000001] 0 3 -3
[1.1000000000000001] 0 2 -4
[1.1000000000000001] 0 1 -5
[2.8999999999999999, 0] 1 1 1
[2.8999999999999999, 0.29999999999999999] 1 0 0
[2.8999999999999999, 0.5] 1 5 -1
[2.8999999999999999, 2.0] 1 4 -2
[2.8999999999999999, 2.0] 1 3 -3
[2.8999999999999999, 2.0] 1 2 -4
[2.8999999999999999, 2.0, 0] 2 2 2
[2.8999999999999999, 2.0, 0.0] 2 1 1
[2.8999999999999999, 2.0, 0.59999999999999998] 2 0 0
[2.8999999999999999, 2.0, 0.90000000000000002] 2 5 -1
[2.8999999999999999, 2.0, 2.8999999999999999] 2 4 -2
[2.8999999999999999, 2.0, 2.8999999999999999] 2 3 -3
[2.8999999999999999, 2.0, 2.8999999999999999, 0] 3 3 3
[2.8999999999999999, 2.0, 2.8999999999999999, 0.0] 3 2 2
[2.8999999999999999, 2.0, 2.8999999999999999, 0.0] 3 1 1
[2.8999999999999999, 2.0, 2.8999999999999999, 0.89999999999999991] 3 0 0
[2.8999999999999999, 2.0,

array([ 2.9,  2. ,  2.9,  3.8,  4.7,  2.6])

In [52]:
w

array([[ 0.11194521,  0.2500605 ,  0.23641504,  0.19122924,  0.11764689,
         0.09270298]], dtype=float32)

In [75]:
ln.circulant([-2, -1, 0, 1, 2])

array([[-2,  2,  1,  0, -1],
       [-1, -2,  2,  1,  0],
       [ 0, -1, -2,  2,  1],
       [ 1,  0, -1, -2,  2],
       [ 2,  1,  0, -1, -2]])